In [8]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'../../../')

import lxmls
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc
from lxmls.readers import pos_corpus

from lxmls.sequences import structured_perceptron 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
corpus = lxmls.readers.pos_corpus.PostagCorpus()
data_path = "../../../../lxmls-toolkit/data/"

train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=10, max_nr_sent=1000)

test_seq = corpus.read_sequence_list_conll(data_path + "test-23.conll", 
                                           max_sent_len=10, max_nr_sent=1000)

dev_seq = corpus.read_sequence_list_conll(data_path + "dev-22.conll", 
                                          max_sent_len=10, max_nr_sent=1000)

In [3]:
## Build features 
feature_mapper = idfc.IDFeatures(train_seq)
feature_mapper.build_features()

In [9]:
## Train the StructuredPerceptron implemented previously
sp = structured_perceptron.StructuredPerceptron(corpus.word_dict, 
                                                corpus.tag_dict,
                                                feature_mapper)
sp.num_epochs = 20
sp.train_supervised(train_seq)

Epoch: 0 Accuracy: 0.656806
Epoch: 1 Accuracy: 0.820898
Epoch: 2 Accuracy: 0.879176
Epoch: 3 Accuracy: 0.907432
Epoch: 4 Accuracy: 0.925239
Epoch: 5 Accuracy: 0.939956
Epoch: 6 Accuracy: 0.946284
Epoch: 7 Accuracy: 0.953790
Epoch: 8 Accuracy: 0.958499
Epoch: 9 Accuracy: 0.955114
Epoch: 10 Accuracy: 0.959235
Epoch: 11 Accuracy: 0.968065
Epoch: 12 Accuracy: 0.968212
Epoch: 13 Accuracy: 0.966740
Epoch: 14 Accuracy: 0.971302
Epoch: 15 Accuracy: 0.968653
Epoch: 16 Accuracy: 0.970419
Epoch: 17 Accuracy: 0.971891
Epoch: 18 Accuracy: 0.971744
Epoch: 19 Accuracy: 0.973510


In [10]:
## Evaluate the StructuredPerceptron model

pred_train = sp.viterbi_decode_corpus(train_seq)
pred_dev = sp.viterbi_decode_corpus(dev_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)

eval_train = sp.evaluate_corpus(train_seq, pred_train)
eval_dev = sp.evaluate_corpus(dev_seq, pred_dev)
eval_test = sp.evaluate_corpus(test_seq, pred_test)

print("Structured Perceptron - ID Features Accuracy Train: %.3f Dev: %.3f Test: %.3f"\
      %( eval_train,eval_dev,eval_test))

Structured Perceptron - ID Features Accuracy Train: 0.984 Dev: 0.835 Test: 0.840


### Training the model with the new feature set

In [11]:
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()


In [12]:
sp = structured_perceptron.StructuredPerceptron(corpus.word_dict,
                                                corpus.tag_dict, 
                                                feature_mapper)
sp.num_epochs = 20
sp.train_supervised(train_seq)

Epoch: 0 Accuracy: 0.764386
Epoch: 1 Accuracy: 0.872701
Epoch: 2 Accuracy: 0.903458
Epoch: 3 Accuracy: 0.927594
Epoch: 4 Accuracy: 0.938484
Epoch: 5 Accuracy: 0.951141
Epoch: 6 Accuracy: 0.949816
Epoch: 7 Accuracy: 0.959529
Epoch: 8 Accuracy: 0.957616
Epoch: 9 Accuracy: 0.962325
Epoch: 10 Accuracy: 0.961148
Epoch: 11 Accuracy: 0.970567
Epoch: 12 Accuracy: 0.968212
Epoch: 13 Accuracy: 0.973216
Epoch: 14 Accuracy: 0.974393
Epoch: 15 Accuracy: 0.973951
Epoch: 16 Accuracy: 0.976600
Epoch: 17 Accuracy: 0.977483
Epoch: 18 Accuracy: 0.974834
Epoch: 19 Accuracy: 0.977042


In [13]:
## Evaluate the StructuredPerceptron model with the extended features

pred_train = sp.viterbi_decode_corpus(train_seq)
pred_dev = sp.viterbi_decode_corpus(dev_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)

eval_train = sp.evaluate_corpus(train_seq, pred_train)
eval_dev = sp.evaluate_corpus(dev_seq, pred_dev)
eval_test = sp.evaluate_corpus(test_seq, pred_test)

In [26]:
print("SP_ext -  Accuracy Train: %.3f Dev: %.3f Test: %.3f"\
      %(eval_train,eval_dev, eval_test))


SP_ext -  Accuracy Train: 0.984 Dev: 0.888 Test: 0.890


#### Summary of the results in the exercises 

| Model   | Train acc | Dev acc | Test acc |
| --------| --------- | ------- |--------- |
| crf     |  0.949    | 0.846   |   0.858  |
| crf_ext |  0.984    | 0.899   |   0.894  |
| sp      |  0.984    | 0.835   |   0.840  |
| sp_ext  |  0.984    | 0.888   |   0.890  |